# Task 1: Exploratory Data Analysis and Data Preprocessing

## Objective
Understand the structure, content, and quality of the complaint data and prepare it for the RAG pipeline.

## Checklist
1. Load Raw CFPB Dataset
2. Initial Exploratory Data Analysis (EDA)
3. Filter to Business Scope
4. Text Cleaning & Normalization
5. Save Cleaned Dataset

In [ ]:
# 1. Environment Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re

# Configuration
RAW_DATA_PATH = '../data/raw/complaints.csv'
PROCESSED_DATA_PATH = '../data/processed/filtered_complaints.csv'

# Ensure directores exist
os.makedirs('../data/processed', exist_ok=True)
os.makedirs('eda_outputs', exist_ok=True)

## 2. Load Raw CFPB Dataset
Loading the dataset efficiently. Since the file is large, we will use chunking or load a sample first for EDA.

In [ ]:
# Load a sample to inspect columns and types
df_sample = pd.read_csv(RAW_DATA_PATH, nrows=5)
print("Shape (Sample):", df_sample.shape)
print("Columns:", df_sample.columns.tolist())
print("\nData Types:")
print(df_sample.dtypes)

df_sample.head()

## 3. Initial Exploratory Data Analysis (EDA)
We will iterate through the dataset to calculate aggregate statistics without loading it all into memory.

In [ ]:
product_counts = {}
narrative_counts = {'total': 0, 'with_narrative': 0, 'null_narrative': 0}
narrative_lengths = []

chunk_size = 100000

print("Processing dataset for EDA...")
for chunk in pd.read_csv(RAW_DATA_PATH, chunksize=chunk_size, low_memory=False):
    # Product Distribution
    if 'Product' in chunk.columns:
        counts = chunk['Product'].value_counts()
        for prod, count in counts.items():
            product_counts[prod] = product_counts.get(prod, 0) + count
    
    # Narrative Availability
    if 'Consumer complaint narrative' in chunk.columns:
        chunk_total = len(chunk)
        narrative_counts['total'] += chunk_total
        non_null = chunk['Consumer complaint narrative'].notnull().sum()
        narrative_counts['with_narrative'] += non_null
        narrative_counts['null_narrative'] += (chunk_total - non_null)
        
        # Length of non-null narratives
        valid = chunk['Consumer complaint narrative'].dropna()
        lengths = valid.astype(str).apply(lambda x: len(x.split())).tolist()
        narrative_lengths.extend(lengths)
        
print("EDA Processing Complete.")

### 3.1 Product Distribution

In [ ]:
prod_df = pd.DataFrame(list(product_counts.items()), columns=['Product', 'Count']).sort_values('Count', ascending=False)

plt.figure(figsize=(12, 6))
sns.barplot(data=prod_df.head(15), x='Count', y='Product')
plt.title('Top 15 Products by Complaint Volume')
plt.show()

### 3.2 Narrative Availability

In [ ]:
print("Narrative Counts:", narrative_counts)
missing_pct = (narrative_counts['null_narrative'] / narrative_counts['total']) * 100
print(f"Percentage of complaints without narrative: {missing_pct:.2f}%")

### 3.3 Narrative Length Analysis

In [ ]:
lengths_series = pd.Series(narrative_lengths)
print(lengths_series.describe())

plt.figure(figsize=(10, 5))
sns.histplot(narrative_lengths, bins=50, kde=True)
plt.title('Distribution of Complaint Narrative Lengths (Word Count)')
plt.xlabel('Word Count')
plt.xlim(0, 1000)
plt.show()

## 4. Filter to Business Scope & 5. Text Cleaning
Filtering for: Credit card, Personal loan, Savings account, Money transfers.
Removing empty narratives.
Cleaning text.

In [ ]:
def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = text.replace('xx/xx/xxxx', '')
    text = text.replace('xxxx', '')
    text = re.sub(r'\s+', ' ', text).strip()
    return text

keywords = ['credit card', 'personal loan', 'savings', 'money transfer']

processed_chunks = []

print("Filtering and cleaning...")
for chunk in pd.read_csv(RAW_DATA_PATH, chunksize=chunk_size, low_memory=False):
    if 'Product' not in chunk.columns or 'Consumer complaint narrative' not in chunk.columns:
        continue
        
    chunk['Product_Normalized'] = chunk['Product'].astype(str).str.lower()
    
    mask_product = chunk['Product_Normalized'].apply(lambda x: any(k in x for k in keywords))
    mask_narrative = chunk['Consumer complaint narrative'].notnull()
    
    filtered_chunk = chunk[mask_product & mask_narrative].copy()
    
    if len(filtered_chunk) > 0:
        filtered_chunk['cleaned_narrative'] = filtered_chunk['Consumer complaint narrative'].apply(clean_text)
        
        rename_map = {
            'Complaint ID': 'complaint_id',
            'Product': 'product',
            'Issue': 'issue',
            'Sub-issue': 'sub_issue',
            'Company': 'company',
            'Date received': 'date_received'
        }
        
        cols_to_keep = list(rename_map.keys()) + ['cleaned_narrative']
        existing_cols = [c for c in cols_to_keep if c in filtered_chunk.columns]
        
        final_chunk = filtered_chunk[existing_cols].rename(columns=rename_map)
        processed_chunks.append(final_chunk)

final_df = pd.concat(processed_chunks, ignore_index=True)
print(f"Final Dataset Shape: {final_df.shape}")

## 6. Save Cleaned Dataset

In [ ]:
final_df.to_csv(PROCESSED_DATA_PATH, index=False)
print(f"Saved to {PROCESSED_DATA_PATH}")

# Verification
print(final_df.info())